In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pandas scipy statsmodels

In [5]:
import pandas as pd

df_ab = pd.read_csv('/content/drive/MyDrive/Womakers/experimento_teste_ab.csv')
df_ab.head()

,Visitante_ID,Versão_Página,Visualizações,Conversões
0,1,A,167,1
1,2,A,160,0
2,3,A,170,0
3,4,A,161,1
4,5,A,104,0


1. Qual dos cenários tem a maior taxa de conversão?

In [6]:
df_ab['taxa_conversao'] = df_ab['Conversões'] / df_ab['Visualizações'] * 100
maior_taxa = df_ab.loc[df_ab['taxa_conversao'].idxmax()]

print(f"Cenário com a maior taxa de conversão: {maior_taxa['Versão_Página']} com {maior_taxa['taxa_conversao']:.2f}%")

Cenário com a maior taxa de conversão: A com 1.00%


2. Calcule qual o tamanho da amostra necessária para o desenvolvimento
de um teste A/B, seguindo os seguintes critérios:

a. O cenário A, da base, como o inicial, que funciona hoje.

b. Considere 95% de confiança de que o efeito na conversão não foi
aleatório com um nível de significância de 5% (alpha = 0,05).

c. Também considere 80% de certeza conseguir capturar o efeito
da nova abordagem.

d. O aumento para 10% de conversão.


In [7]:
import scipy.stats as stats

p1 = df_ab.loc[df_ab['Versão_Página'] == 'A', 'taxa_conversao'].mean() / 100  # taxa de conversão atual
p2 = p1 + 0.1 * p1
alpha = 0.05
power = 0.80

z_alpha = stats.norm.ppf(1 - alpha / 2)
z_beta = stats.norm.ppf(power)

n = ((z_alpha**2 * p1 * (1 - p1)) + (z_beta**2 * p2 * (1 - p2))) / (p2 - p1)**2
n = round(n)
print(f"Tamanho da amostra necessária para o teste A/B: {n}")


Tamanho da amostra necessária para o teste A/B: 660534


Verifique a base pacientes.csv, os dados são fictícios. A sintetiza uma base
de dados de um hospital que trata de pacientes com problemas cardíacos.
Considere a base como a população, portanto as estatísticas da população
são conhecidas. Responda as perguntas abaixo:


3. Considerando uma amostra de 45 números que representam o index do
dataframe, índices= ([909, 751, 402, 400, 726, 39, 184, 269, 255, 769,
209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967,
699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348,
146, 505, 273, 957, 362]). Considerando essa amostra é possível dizer
que a idade média das pessoas com problemas cardíacos é maior que
50 anos? Nível de significância igual a 5%.


In [9]:
df_pacientes = pd.read_csv('/content/drive/MyDrive/Womakers/pacientes.csv')
df_pacientes.head()

,Unnamed: 0,Pressao_Arterial,Idade,Genero,Etnia,Estado_Saude,Nome_Genero,Nome_Etnia,Nome_Estado_Saude
0,0,142.755809,31,1,1,1,Feminino,Afro-americano,Com condições de saúde adicionais
1,1,161.878468,35,1,1,0,Feminino,Afro-americano,Saudável
2,2,161.508259,39,0,3,1,Masculino,Asiático,Com condições de saúde adicionais
3,3,144.979408,43,1,3,1,Feminino,Asiático,Com condições de saúde adicionais
4,4,162.502954,55,0,2,1,Masculino,Hispânico,Com condições de saúde adicionais


In [11]:
indices = [909, 751, 402, 400, 726, 39, 184, 269, 255, 769, 209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967, 699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348, 146, 505, 273, 957, 362]
amostra_idade = df_pacientes.loc[indices, 'Idade']

media_observada = amostra_idade.mean()
t_stat, p_value = stats.ttest_1samp(amostra_idade, 50)

print(f"Média da amostra: {media_observada:.2f}, Valor p: {p_value:.4f}")
if p_value < 0.05:
    print("Rejeita H0: A idade média é maior que 50 anos.")
else:
    print("Não rejeita H0: A idade média não é maior que 50 anos.")

Média da amostra: 50.09, Valor p: 0.9585
Não rejeita H0: A idade média não é maior que 50 anos.


4. Queremos entender que tipo de amostra estamos lidando se dividirmos
os conjuntos em 2, sendo um com pessoas que têm condições de
saúde adicionais e o outro com pessoas saudáveis. Seria dependente
ou independente?

Resposta: Amostras Independentes.

5. Agora considere o um conjunto de pessoas aleatória que representam
o index do dataframe, índices = ([690, 894, 67, 201, 364, 19, 60, 319,
588, 643, 855, 623, 530, 174, 105, 693, 6, 462, 973, 607, 811, 346, 354,
966, 943, 372]), podemos dizer que a pressão arterial média para
pacientes com condições de saúde adicionais é igual à pressão arterial
média para pacientes sem condições adicionais de saúde? Considere o
nível de significância a 6%.

In [27]:
grupo_saude_adicional = amostra[amostra['Nome_Estado_Saude'] == 'Com condições de saúde adicionais']['Pressao_Arterial']
grupo_saude_normal = amostra[amostra['Nome_Estado_Saude'] == 'Saudável']['Pressao_Arterial']

n_adicional = grupo_saude_adicional.count()
n_normal = grupo_saude_normal.count()

if n_adicional > 0 and n_normal > 0:
    t_stat, p_val = stats.ttest_ind(grupo_saude_adicional, grupo_saude_normal, nan_policy='omit')
    print(f"Valor p do teste: {p_val}")

    alpha = 0.06
    if p_val < alpha:
        print("Rejeita H0: A pressão arterial média é diferente entre os grupos.")
    else:
        print("Não rejeita H0: A pressão arterial média é igual entre os grupos.")
else:
    print("Uma ou ambas as amostras estão vazias. Não é possível realizar o teste.")


Valor p do teste: 0.9359246016248557
Não rejeita H0: A pressão arterial média é igual entre os grupos.


6. Existe uma diferença significativa na pressão arterial média entre
diferentes grupos étnicos nesta população? (Teste ANOVA, alpha é 5%)

a. Hipótese Nula (H0): A pressão arterial média é a mesma em todos
os grupos étnicos.

b. Hipótese Alternativa (H1): Há uma diferença significativa na
pressão arterial média entre pelo menos dois grupos étnicos
nesta população

In [13]:
grupos = df_pacientes.groupby('Etnia')['Pressao_Arterial'].apply(list)  # Usar a coluna 'Pressao_Arterial'

f_stat, p_value = stats.f_oneway(*grupos)

print(f"Valor p do teste ANOVA: {p_value:.4f}")
if p_value < 0.05:
    print("Rejeita H0: Há diferença significativa na pressão arterial entre grupos étnicos.")
else:
    print("Não rejeita H0: Não há diferença significativa na pressão arterial entre grupos étnicos.")


Valor p do teste ANOVA: 0.8738
Não rejeita H0: Não há diferença significativa na pressão arterial entre grupos étnicos.


7. Lúcia é uma pesquisadora e tem o objetivo de entender a relação de
gênero neste grupo de pacientes. Acredita-se que há uma relação
entre o sexo e condições de saúde adicionais. (Teste qui-quadrado)


In [16]:
tabela_contingencia = pd.crosstab(df_pacientes['Genero'], df_pacientes['Estado_Saude'])

chi2, p_value, dof, expected = stats.chi2_contingency(tabela_contingencia)

print(f"Valor p do teste Qui-quadrado: {p_value:.4f}")
if p_value < 0.05:
    print("Rejeita H0: Existe uma relação entre gênero e estado de saúde.")
else:
    print("Não rejeita H0: Não há relação entre gênero e estado de saúde.")


Valor p do teste Qui-quadrado: 1.0000
Não rejeita H0: Não há relação entre gênero e estado de saúde.


8. Existe uma associação entre a idade dos pacientes e sua pressão
arterial?
a. Hipótese Nula (H0): se a pressão arterial é independente da idade
b. Hipótese Alternativa (H1): a pressão arterial esta associada a
idade

In [17]:
correlacao, p_value = stats.pearsonr(df_pacientes['Idade'], df_pacientes['Pressao_Arterial'])
print(f"Coeficiente de correlação: {correlacao:.4f}, Valor p: {p_value:.4f}")
if p_value < 0.05:
    print("Rejeita H0: A pressão arterial está associada à idade.")
else:
    print("Não rejeita H0: A pressão arterial é independente da idade.")


Coeficiente de correlação: -0.0214, Valor p: 0.4994
Não rejeita H0: A pressão arterial é independente da idade.


9. Qual é o intervalo de confiança para a média da pressão arterial entre
os pacientes com condições de saúde adicionais? (nível de confiança
95%)


In [21]:
media_pressao = df_pacientes['Pressao_Arterial'].mean()
desvio_padrao = df_pacientes['Pressao_Arterial'].std()
n = df_pacientes['Pressao_Arterial'].count()
z = stats.norm.ppf(0.975)

erro_padrao = desvio_padrao / (n**0.5)
intervalo_conf = (media_pressao - z * erro_padrao, media_pressao + z * erro_padrao)

print(f"Intervalo de confiança para a média da pressão arterial: {intervalo_conf}")


Intervalo de confiança para a média da pressão arterial: (149.75932244741375, 150.9919227836313)


10. A distribuição da pressão arterial na população segue uma distribuição
normal?

a. Hipótese Nula (H0): A distribuição da pressão arterial na
população segue uma distribuição normal.

b. Hipótese Alternativa (H1): A distribuição da pressão arterial na
população não segue uma distribuição normal.

In [22]:
stat, p_value = stats.shapiro(df_pacientes['Pressao_Arterial'])

print(f"Valor p do teste de normalidade: {p_value:.4f}")
if p_value < 0.05:
    print("Rejeita H0: A distribuição da pressão arterial não segue uma distribuição normal.")
else:
    print("Não rejeita H0: A distribuição da pressão arterial segue uma distribuição normal.")


Valor p do teste de normalidade: 0.0607
Não rejeita H0: A distribuição da pressão arterial segue uma distribuição normal.
